In [ ]:
s3_uri = "asimtesting/nuplan_private_test/2021.07.25.16.16.23_veh-26_02446_02589.arrow"

In [ ]:

import pyarrow as pa
import pyarrow.fs as fs
import pyarrow.dataset as ds

import os

s3_fs = fs.S3FileSystem(
    access_key=os.environ.get('AWS_ACCESS_KEY_ID'),
    secret_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
    region=os.environ.get('AWS_DEFAULT_REGION')
)
from asim.common.utils.timer import Timer


timer = Timer()
timer.start()

dataset = ds.dataset(f"{s3_uri}", format="ipc", filesystem=s3_fs)
timer.log("1. Dataset loaded")

# Get all column names and remove the ones you want to drop
all_columns = dataset.schema.names
columns_to_keep = [col for col in all_columns if col not in ["front_cam_demo", "front_cam_transform"]]
timer.log("2. Columns filtered")

table = dataset.to_table(columns=columns_to_keep)
timer.log("3. Table created")
# Save locally
with pa.ipc.new_file("filtered_file.arrow", table.schema) as writer:
    writer.write_table(table)
timer.log("4. Table saved locally")

timer.end()
timer.stats(verbose=False)

In [ ]:
timer = Timer()
timer.start()

table = dataset.take([i for i in range(0, 100, 1)], columns=all_columns)
timer.log("3. Table created")

# Save locally
with pa.ipc.new_file("filtered_file_v2.arrow", table.schema) as writer:
    writer.write_table(table)

timer.log("4. Table saved locally")
timer.end()
timer.stats(verbose=False)


In [ ]:
# with pa.memory_map("filtered_file_v2.arrow", "r") as source:
#     mmap_table = pa.ipc.open_file(source).read_all()

In [ ]:
# import pyarrow.parquet as pq
# import pyarrow.dataset as ds
# import pyarrow.dataset as ds
# import pyarrow as pa
# from huggingface_hub import hf_hub_download
# from io import BytesIO

# # Download specific file from the repo
# file_path = hf_hub_download(
#     repo_id="DanielDauner/delete_me",
#     filename="2021.05.25.14.16.10_veh-35_01690_02183.arrow",  # or whatever the file is named
#     repo_type="dataset"
# )

# # Read with PyArrow
# with pa.memory_map(file_path, "r") as source:
#     mmap_table = pa.ipc.open_file(source).read_all()

# # Create dataset and slice
# sliced_table = mmap_table.slice(0, 1000)  # First 1000 rows
# sliced_table